In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
max_profit = 0
min_loss = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [3]:
symbols = """
select distinct(name) from stockdatadaily order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [5]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = 0.0
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    training = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatadaily order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [6]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatadaily where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done

In [7]:
sql_dates = """
select distinct(date) from stockdatadaily order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

In [8]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

<h1> Loading ANN model </h1>

In [9]:
model = tf.keras.models.load_model('61-58DailyANN.model', custom_objects={'sensitivity': sensitivity})

<h1> Trading strategy with the training sets generation

In [10]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global max_profit
    global min_loss
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatadaily where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        training_inst = training_set_extractor(data_till_date, date)
                        for_predict = np.asarray(tf.keras.utils.normalize([[training_inst[0], training_inst[1]]]))
                        if(model.predict(for_predict) > 0.5):
                            buy_stocks(current_price, current_symbol, date)
        
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                
                                win += 1
                                current_symbol.flag_for_wins = True
                                
                        
                        
                    if current_symbol.stop_order > current_price:
                                             
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            
                            if current_symbol.flag_for_wins == False:
                                win += 1
                                
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1
                            
                            current_symbol.flag_for_wins = False
                    
                            
                        profit += profit_today
                        
                        if profit > max_profit:
                            max_profit = profit
                        if profit < min_loss:
                            min_loss = profit
                        
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [11]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support



def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global commission
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)

    else:
        pass

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    lastFrom = pd.to_datetime(date)

    destDate = lastFrom - pd.Timedelta(days=30)
    lastFrom = str(lastFrom.date())
    destDate = str(destDate.date())

    df = pd.DataFrame()
    df = data[(data['date'] >= destDate) & (data['date'] <= lastFrom)]

    df = df.set_index('date')

    df.index = pd.DatetimeIndex(df.index)
    idx = pd.date_range(destDate, lastFrom)
    df = df.reindex(idx, fill_value=0)
    
    highs = df['2. high'].tolist()
    volumes = df['5. volume'].tolist()
    lows = df['3. low'].tolist()
    
    for i in range(1, len(highs)):
        if highs[i] == 0:
            highs[i] = highs[i-1]
        if volumes[i] == 0:
            volumes[i] = volumes[i-1]
        if lows[i] == 0:
            lows[i] = lows[i-1]
        
    
    training_instance = [highs, volumes, lows, 'na']
    
    return training_instance[:] 
   

In [12]:
automatic_trader(symbol_list)

Buying  132.0 stocks of  ENB At price 20.5 on date 2000-01-07
Buying  173.0 stocks of  NTIC At price 7.875 on date 2000-01-07
Buying  32.0 stocks of  HIBB At price 18.25 on date 2000-01-10
Buying  530.0 stocks of  AXGN At price 1.656 on date 2000-01-11
Buying  18.0 stocks of  IMMU At price 12.81 on date 2000-01-11
Buying  34.0 stocks of  MYE At price 15.88 on date 2000-01-11
Buying  94.0 stocks of  NOM At price 14.0 on date 2000-01-11
Buying  133.0 stocks of  RGLD At price 3.688 on date 2000-01-11
Buying  29.0 stocks of  ROL At price 15.44 on date 2000-01-11
Buying  38.0 stocks of  USAK At price 8.25 on date 2000-01-11
Buying  5.0 stocks of  XLV At price 32.53 on date 2000-01-11
Buying  2.0 stocks of  AET At price 58.31 on date 2000-01-12
Buying  4.0 stocks of  ASH At price 34.13 on date 2000-01-12
Buying  3.0 stocks of  BLX At price 25.13 on date 2000-01-12
Buying  7.0 stocks of  BSAC At price 22.0 on date 2000-01-12
Buying  3.0 stocks of  CDZI At price 10.75 on date 2000-01-12
Buying

Buying  3.0 stocks of  BKT At price 6.063 on date 2000-04-28
Buying  2.0 stocks of  EVJ At price 11.19 on date 2000-04-28
Selling  18.0 stocks of IMMU  at price 14.81 on date 2000-04-28
Buying  3.0 stocks of  INO At price 4.75 on date 2000-04-28
Buying  4.0 stocks of  LCUT At price 7.25 on date 2000-04-28
Buying  3.0 stocks of  MUJ At price 12.5 on date 2000-04-28
Buying  3.0 stocks of  SLP At price 3.0 on date 2000-04-28
Buying  5.0 stocks of  UUU At price 2.38 on date 2000-04-28
Buying  2.0 stocks of  ABEV At price 10.75 on date 2000-05-01
Buying  2.0 stocks of  NCA At price 8.438 on date 2000-05-01
Buying  2.0 stocks of  ADC At price 14.88 on date 2000-05-02
Selling  6.0 stocks of SGBX  at price 0.1094 on date 2000-05-03
Buying  6.0 stocks of  HNRG At price 1.875 on date 2000-05-10
Buying  12.0 stocks of  ENSV At price 0.437 on date 2000-05-19
Selling  2.0 stocks of IVAC  at price 3.5 on date 2000-05-19
Selling  133.0 stocks of RGLD  at price 3.0 on date 2000-05-19
Buying  2.0 stock

Selling  2.0 stocks of LDF  at price 7.79 on date 2001-10-08
Buying  2.0 stocks of  MMV At price 14.54 on date 2001-11-09
Selling  6.0 stocks of CASM  at price 0.58 on date 2001-11-16
Selling  2.0 stocks of DGSE  at price 2.05 on date 2001-11-29
Selling  19.0 stocks of VTNR  at price 0.01 on date 2001-12-11
Selling  3.0 stocks of RELV  at price 1.2 on date 2001-12-24
Selling  2.0 stocks of NATH  at price 3.35 on date 2001-12-26
Selling  2.0 stocks of CUZ  at price 24.33 on date 2001-12-27
Selling  2.0 stocks of CDE  at price 0.78 on date 2001-12-28
Selling  2.0 stocks of EFOI  at price 0.33 on date 2002-01-10
Buying  2.0 stocks of  IF At price 1.78 on date 2002-01-28
Selling  3.0 stocks of BLX  at price 23.2 on date 2002-01-29
Buying  13.0 stocks of  HUSA At price 0.53 on date 2002-01-30
Buying  3.0 stocks of  EVY At price 14.41 on date 2002-02-04
Selling  530.0 stocks of AXGN  at price 1.88 on date 2002-02-07
Buying  3.0 stocks of  BGC At price 15.75 on date 2002-02-07
Buying  5.0 sto

Buying  2.0 stocks of  MSD At price 8.97 on date 2003-06-05
Selling  3.0 stocks of HA  at price 0.8 on date 2003-07-21
Selling  2.0 stocks of NCA  at price 9.19 on date 2003-08-29
Buying  7.0 stocks of  PLG At price 0.54 on date 2003-09-08
Selling  2.0 stocks of EIP  at price 14.11 on date 2003-09-16
Buying  2.0 stocks of  HALL At price 0.85 on date 2003-09-16
Buying  2.0 stocks of  APLS At price 0.6 on date 2003-09-18
Buying  21.0 stocks of  UVE At price 0.06 on date 2003-10-20
Selling  7.0 stocks of JCTCF  at price 4.9 on date 2003-11-26
Selling  3.0 stocks of CCA  at price 15.6 on date 2003-11-28
Buying  2.0 stocks of  IHG At price 9.05 on date 2003-11-28
Buying  2.0 stocks of  MZF At price 15.2 on date 2003-11-28
Selling  2.0 stocks of WABC  at price 51.06 on date 2003-12-08
Buying  2.0 stocks of  BBL At price 16.84 on date 2003-12-09
Buying  2.0 stocks of  MAV At price 15.14 on date 2003-12-09
Selling  2.0 stocks of DKS  at price 48.31 on date 2003-12-10
Buying  2.0 stocks of  FFC

Selling  2.0 stocks of NTP  at price 12.53 on date 2006-09-15
Buying  2.0 stocks of  FPX At price 19.85 on date 2006-10-05
Selling  2.0 stocks of PAR  at price 7.76 on date 2006-10-09
Selling  10.0 stocks of HFWA  at price 24.81 on date 2006-10-26
Buying  4.0 stocks of  HMSY At price 12.26 on date 2006-10-26
Buying  2.0 stocks of  LTC At price 24.68 on date 2006-10-26
Buying  3.0 stocks of  PEZ At price 26.15 on date 2006-10-26
Buying  2.0 stocks of  PSL At price 25.51 on date 2006-10-26
Selling  2.0 stocks of PKB  at price 15.79 on date 2006-11-13
Selling  2.0 stocks of SGMS  at price 28.29 on date 2006-11-13
Buying  3.0 stocks of  CPRX At price 6.25 on date 2006-11-14
Buying  3.0 stocks of  CRWS At price 4.15 on date 2006-11-15
Buying  2.0 stocks of  LSBK At price 11.89 on date 2006-11-15
Buying  2.0 stocks of  PFM At price 17.0 on date 2006-11-22
Selling  3.0 stocks of TCFC  at price 23.7 on date 2006-11-29
Buying  3.0 stocks of  EWS At price 10.98 on date 2006-11-30
Selling  3.0 st

Selling  2.0 stocks of GCC  at price 32.0 on date 2008-03-20
Selling  4.0 stocks of PFIS  at price 25.75 on date 2008-03-28
Selling  2.0 stocks of ICON  at price 17.66 on date 2008-03-31
Selling  3.0 stocks of SEED  at price 5.7 on date 2008-03-31
Selling  2.0 stocks of SENEB  at price 21.0 on date 2008-04-02
Buying  2.0 stocks of  ECH At price 54.05 on date 2008-04-04
Buying  3.0 stocks of  LL At price 11.41 on date 2008-04-04
Buying  2.0 stocks of  PSTI At price 2.05 on date 2008-04-07
Buying  5.0 stocks of  EBSB At price 10.0 on date 2008-04-08
Selling  3.0 stocks of BCOM  at price 10.53 on date 2008-04-10
Selling  4.0 stocks of LMAT  at price 4.52 on date 2008-04-23
Buying  2.0 stocks of  ADZ At price 26.51 on date 2008-04-25
Selling  5.0 stocks of CNTY  at price 3.29 on date 2008-05-05
Selling  2.0 stocks of SNMX  at price 5.78 on date 2008-05-05
Selling  2.0 stocks of TXMD  at price 0.51 on date 2008-05-05
Selling  2.0 stocks of STRS  at price 24.92 on date 2008-05-19
Buying  2.0

Buying  3.0 stocks of  ZSL At price 29.966 on date 2008-12-12
Buying  6.0 stocks of  AGZ At price 107.47 on date 2008-12-15
Buying  16.0 stocks of  SGYP At price 2.45 on date 2008-12-16
Buying  2.0 stocks of  CEL At price 21.47 on date 2008-12-22
Selling  2.0 stocks of EGI  at price 0.9 on date 2008-12-22
Selling  2.0 stocks of LDF  at price 9.42 on date 2008-12-22
Selling  2.0 stocks of PBD  at price 12.49 on date 2008-12-22
Selling  2.0 stocks of PFM  at price 11.37 on date 2008-12-22
Selling  2.0 stocks of RJI  at price 5.9716 on date 2008-12-22
Selling  2.0 stocks of USL  at price 33.046 on date 2008-12-22
Selling  2.0 stocks of ATAI  at price 5.08 on date 2008-12-23
Selling  3.0 stocks of KS  at price 2.88 on date 2009-01-05
Selling  12.0 stocks of PMO  at price 9.1 on date 2009-01-05
Selling  2.0 stocks of CNS  at price 13.46 on date 2009-01-06
Selling  3.0 stocks of CWI  at price 23.8304 on date 2009-01-06
Selling  2.0 stocks of DWSN  at price 2.8 on date 2009-01-06
Selling  4.0

Buying  2.0 stocks of  HSEB At price 28.1 on date 2010-11-18
Buying  5.0 stocks of  IOVA At price 1.25 on date 2010-12-31
Buying  2.0 stocks of  CMRE At price 15.805 on date 2011-02-04
Selling  2.0 stocks of NBB  at price 17.99 on date 2011-02-04
Buying  9.0 stocks of  XXII At price 1.4 on date 2011-02-28
Selling  45.0 stocks of CALI  at price 1.86 on date 2011-04-20
Buying  2.0 stocks of  SPE At price 15.79 on date 2011-04-20
Selling  2.0 stocks of ADZ  at price 22.53 on date 2011-04-25
Buying  7.0 stocks of  GCVRZ At price 2.47 on date 2011-04-25
Buying  2.0 stocks of  BEP At price 24.58 on date 2011-05-02
Buying  3.0 stocks of  SELF At price 4.48 on date 2011-05-06
Buying  4.0 stocks of  ONCS At price 1.68 on date 2011-05-10
Selling  6.0 stocks of GXG  at price 20.98 on date 2011-05-16
Buying  6.0 stocks of  HEAR At price 0.72 on date 2011-05-16
Buying  10.0 stocks of  SPHS At price 0.62 on date 2011-05-16
Buying  2.0 stocks of  CBND At price 30.97 on date 2011-05-17
Selling  8.0 st

Buying  2.0 stocks of  PXMV At price 24.622 on date 2013-12-24
Selling  2.0 stocks of LAQ  at price 25.86 on date 2014-02-18
Selling  3.0 stocks of ACST  at price 1.26 on date 2014-02-19
Buying  2.0 stocks of  HEWG At price 24.75 on date 2014-02-21
Selling  2.0 stocks of RSXJ  at price 36.78 on date 2014-02-25
Buying  2.0 stocks of  ACSF At price 14.35 on date 2014-02-26
Buying  2.0 stocks of  QINC At price 20.06 on date 2014-03-24
Selling  3.0 stocks of KBWB  at price 34.68 on date 2014-05-15
Buying  2.0 stocks of  HBP At price 4.49 on date 2014-05-20
Buying  2.0 stocks of  FSIC At price 10.46 on date 2014-05-27
Buying  2.0 stocks of  FTHI At price 20.37 on date 2014-05-29
Selling  2.0 stocks of ENSG  at price 31.085 on date 2014-06-05
Buying  2.0 stocks of  MTGEP At price 25.01 on date 2014-06-05
Selling  2.0 stocks of WAIR  at price 19.29 on date 2014-07-28
Buying  2.0 stocks of  TSQ At price 11.59 on date 2014-09-11
Selling  2.0 stocks of BRSS  at price 15.02 on date 2014-09-18
Sel

ValueError: cannot reindex from a duplicate axis

In [ ]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

In [ ]:
remaining_cash